In [3]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston


sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print("Using bucket " + bucket)



Using bucket sagemaker-us-east-1-735287914370
Using bucket sagemaker-us-east-1-735287914370


In [4]:
# we use the Boston housing dataset
data = load_boston()

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX["target"] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX["target"] = y_test



In [6]:
trainX.head()

trainX.to_csv("boston_train.csv")
testX.to_csv("boston_test.csv")

# send data to S3. SageMaker will take training data from s3
trainpath = sess.upload_data(
    path="boston_train.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)

testpath = sess.upload_data(
    path="boston_test.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)



In [5]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":

    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="boston_train.csv")
    parser.add_argument("--test-file", type=str, default="boston_test.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)



Writing script.py


In [6]:
! python script.py --n-estimators 100 \
                   --min-samples-leaf 2 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \
                   --features 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT' \
                   --target target

extracting arguments
reading data
building training and testing datasets
training model
validating model
AE-at-10th-percentile: 0.2345519047619078
AE-at-50th-percentile: 1.6417654761904856
AE-at-90th-percentile: 4.56391595238094
model persisted at ./model.joblib
2


In [7]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",#"ml.c5.xlarge",#"ml.m4.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 3,
        "features": "CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT",
        "target": "target",
    },
)

import time
tic = time.time()

# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=False)

toc = time.time()
print(toc - tic)

0.7027723789215088


In [8]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)

from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

predictor = model.deploy(instance_type="ml.c5.large", initial_instance_count=1)




2022-11-23 02:50:32 Starting - Preparing the instances for training.............
2022-11-23 02:51:50 Downloading - Downloading input data.......
2022-11-23 02:52:31 Training - Downloading the training image.........
2022-11-23 02:53:21 Training - Training image download completed. Training in progress...
2022-11-23 02:53:32 Uploading - Uploading generated training model..
2022-11-23 02:53:49 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-735287914370/rf-scikit-2022-11-23-02-50-02-467/output/model.tar.gz
-----!

In [16]:
from sagemaker.predictor import Predictor
from sagemaker.deserializers import NumpyDeserializer
from sagemaker.serializers import NumpySerializer
predictor = Predictor(endpoint_name="sagemaker-scikit-learn-2022-11-23-02-53-53-264", 
                      sagemaker_session=sess, 
                     serializer=NumpySerializer(), 
                     deserializer=NumpyDeserializer())

In [20]:
predictor.predict(X_test)

array([22.92105174, 30.87948658, 17.04323193, 23.38481035, 16.90950833,
       21.63524739, 19.59596667, 15.89064062, 21.23767546, 21.05423929,
       20.03931126, 19.54032695,  7.98571631, 21.92943451, 19.17768157,
       25.33800833, 18.57025988,  8.75147569, 44.25618218, 15.65557504,
       23.90365931, 23.99051152, 14.86813373, 23.55446105, 15.15907453,
       15.53850397, 21.99884868, 14.08493056, 19.56059372, 20.99159116,
       19.76891861, 23.46285758, 27.94945209, 20.52930996, 14.40185249,
       16.01459805, 34.92382276, 19.07363517, 20.81781515, 24.18591707,
       19.96895509, 29.62202868, 44.55398337, 19.14189405, 22.50385214,
       13.68369333, 15.38115981, 24.05656486, 18.41072082, 28.78148023,
       21.05703056, 33.56258502, 16.89223362, 26.27197175, 45.83636908,
       21.98019104, 15.40095162, 32.63582572, 22.16494978, 20.50763286,
       25.42327381, 34.10356886, 30.26866317, 18.6245127 , 27.35354286,
       17.26636912, 13.44727615, 23.1488428 , 28.83892583, 15.18

array([22.92105174, 30.87948658, 17.04323193, 23.38481035, 16.90950833,
       21.63524739, 19.59596667, 15.89064062, 21.23767546, 21.05423929,
       20.03931126, 19.54032695,  7.98571631, 21.92943451, 19.17768157,
       25.33800833, 18.57025988,  8.75147569, 44.25618218, 15.65557504,
       23.90365931, 23.99051152, 14.86813373, 23.55446105, 15.15907453,
       15.53850397, 21.99884868, 14.08493056, 19.56059372, 20.99159116,
       19.76891861, 23.46285758, 27.94945209, 20.52930996, 14.40185249,
       16.01459805, 34.92382276, 19.07363517, 20.81781515, 24.18591707,
       19.96895509, 29.62202868, 44.55398337, 19.14189405, 22.50385214,
       13.68369333, 15.38115981, 24.05656486, 18.41072082, 28.78148023,
       21.05703056, 33.56258502, 16.89223362, 26.27197175, 45.83636908,
       21.98019104, 15.40095162, 32.63582572, 22.16494978, 20.50763286,
       25.42327381, 34.10356886, 30.26866317, 18.6245127 , 27.35354286,
       17.26636912, 13.44727615, 23.1488428 , 28.83892583, 15.18

In [29]:
total_invocations = 2000
for i in range(total_invocations): 
    predictor.predict(X_test)

In [30]:
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint_name)

{'ResponseMetadata': {'RequestId': '41aba32e-7951-468c-b43a-4580878382fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41aba32e-7951-468c-b43a-4580878382fa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 23 Nov 2022 03:41:54 GMT'},
  'RetryAttempts': 0}}

{'ResponseMetadata': {'RequestId': '41aba32e-7951-468c-b43a-4580878382fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41aba32e-7951-468c-b43a-4580878382fa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 23 Nov 2022 03:41:54 GMT'},
  'RetryAttempts': 0}}